In [1]:
import joblib

def load_model(model_name: str, path="./"):
    model_path = f"{path}{model_name}.joblib"
    loaded_model = joblib.load(model_path)
    return loaded_model

In [2]:
load_model = load_model("sample_model")

In [30]:
import numpy as np
def process_data(loaded_model, predict_len=5):
    # Iterate through the 20 stock data files from /data/db/stock_data/{stock_symbol}.csv
    stock_symbols = ["FGEN", "GLO", "ICT"]

    # Initialize the dictionary that will hold the processed data
    # key = stock symbol
    # value = (last actual cloaing, list of predictions with length=predict_len)
    processed_data = {stock:[] for stock in stock_symbols}

    # Iterate through the stock symbols
    for stock in stock_symbols:
        predicted_closings = []
        # Only get the closing price, which is at the 4th index
        closing_prices = np.array([])
        with open (f"./stock_data/{stock}.csv", "r") as f:
            for line in f:
                try:
                    closing_prices = np.append(closing_prices, float(line.split(",")[4]))
                except ValueError:
                    pass
        last_actual = closing_prices[-1]

        print(f"Closing prices for {stock}: {closing_prices}")
        # From stock_data, use the loaded model to predict a len=predict_len number of predictions
        for _ in range(predict_len):
            print(f"Predicting closing price for {stock}...")
            predicted_closing = loaded_model.predict(closing_prices.reshape(-1, 1))
            print(f"Predicted closing price for {stock}: {predicted_closing[-1]}")
            predicted_closings.append(predicted_closing[-1])
            print(f"Appending predicted closing price for {stock} to closing_prices...")
            closing_prices = np.append(closing_prices, predicted_closing)
        # Append the tiple of last actual closing , and list of predictions 
        #   to the processed_data dictionary
        processed_data[stock] = tuple((last_actual, predicted_closings))
    
    return processed_data

In [31]:
processed_data = process_data(load_model)

Closing prices for FGEN: [27.65 28.25 28.15 28.05 28.1  27.75 27.9  27.75 27.2  27.15 27.2  28.
 27.7  28.   27.95 27.75 27.95 27.5  27.5  27.45 27.2  27.   27.   27.05
 27.05 26.6  26.4  26.25 26.25 27.   27.15 27.9  27.45 27.4  27.35 27.4
 26.05 26.7  26.15 26.6  26.2  25.   24.65 26.75 26.8  26.   25.   24.25
 24.55 24.85 25.6  25.95 26.35 26.   26.4  25.95 26.1  26.35 26.3  25.9
 25.2  25.2  24.85 24.9  25.1  25.2  25.15 25.2  25.3  25.   25.   24.1
 23.9  23.3  23.2  23.3  23.5  23.8  22.6  22.5  20.65 20.1  20.   20.
 19.34 20.45 20.3  19.7  20.   19.3  19.2  19.   19.2  19.2  18.36 19.
 18.8  19.   18.8  18.9  18.   17.8  17.38 16.56 16.8  16.52 17.   17.
 16.98 16.96 17.16 16.96 16.8  17.   16.9  17.   17.04 17.2  17.84 17.6
 17.2  17.2  17.44 17.1  16.64 15.72 16.42 16.98 16.78 17.   17.2  17.08
 17.42 17.2  17.52 17.22 17.24 17.2  17.5  17.22 17.02 17.5  17.1  17.
 17.6  18.   19.34 19.62 19.4  19.4  19.6  19.8  19.24 19.14 18.9  18.58
 18.74 18.5  17.74 18.   18.02 17.3  17.

In [32]:
processed_data

{'FGEN': (17.88,
  [45.380887005822935,
   45.99117262550731,
   45.24789842999049,
   45.7028253518827,
   45.826566777867306]),
 'GLO': (2060.0,
  [45.24712085502283,
   45.70078002094954,
   45.831371515233286,
   45.493715541334026,
   46.08278231387291]),
 'ICT': (203.0,
  [45.24712085502283,
   45.70078002094954,
   45.831371515233286,
   45.493715541334026,
   46.08278231387291])}

In [33]:
def post_processing(processed_data, minimum_score=0.1):
    stock_symbols = ["FGEN", "GLO", "ICT"]

    # Dictionary containing the last acutaul closing price of each stock
    last_actual_closing = {stock:processed_data[stock][0] for stock in stock_symbols}
    
    # Initialize the dictionaries for the stocks to buy and stocks to sell
    stocks_to_buy = {}
    stocks_to_sell = {}

    # Iterate through the processed data
    for stock in stock_symbols:
        predictions = processed_data[stock][1]
        # Calculate the percentile increase form last actual closing to the last prediction
        #   and add it to the stocks_to_buy or stocks_to_sell dictionary
        #   depending on the value of the percentile increase
        percentile_increase = (predictions[-1] - last_actual_closing[stock]) / last_actual_closing[stock]
        if percentile_increase > minimum_score:
            stocks_to_buy[stock] = processed_data[stock]
        else:
            stocks_to_sell[stock] = processed_data[stock]
    
    return stocks_to_buy, stocks_to_sell

In [34]:
stocks_to_buy, stocks_to_sell = post_processing(processed_data)

In [35]:
stocks_to_buy

{'FGEN': (17.88,
  [45.380887005822935,
   45.99117262550731,
   45.24789842999049,
   45.7028253518827,
   45.826566777867306])}

In [36]:
stocks_to_sell

{'GLO': (2060.0,
  [45.24712085502283,
   45.70078002094954,
   45.831371515233286,
   45.493715541334026,
   46.08278231387291]),
 'ICT': (203.0,
  [45.24712085502283,
   45.70078002094954,
   45.831371515233286,
   45.493715541334026,
   46.08278231387291])}